# Collaborative Workflow

The **Collaborative Workflow** allows volunteers to build upon each other’s classifications in a shared, iterative process.
Rather than starting from scratch, each volunteer sees the **previous volunteer’s freehand drawing** and can **edit** it directly.
This approach enables more accurate and consistent annotations for complex or subjective tasks such as freehand polygon drawing.

In a standard Zooniverse workflow, each volunteer’s classification is treated independently.
However, in the collaborative setup:

- The **latest** classification is passed forward as the starting point for the next volunteer.
- Volunteers can then edit existing annotations.

This notebook assumes you are already familiar with completing extractions and reductions. If not, check out the [Polygon data - extraction and reduction.ipynb](https://github.com/zooniverse/aggregation-for-caesar/blob/master/docs/source/Polygon%20data%20-%20extraction%20and%20reduction.ipynb) guide.

Setting up the collaborative workflow involves three key stages:

1. **Reducer Configuration** — defines how Caesar handles and aggregates classifications.
2. **Database Setup on Caesar** — connects the extractor and reducer via external URLs.
3. **Front-End Configuration** — enables the workflow to fetch and display the most recent classifications from Caesar.

Each of these steps is detailed below. Once configured correctly, volunteers will see the **previous classification** when opening a new subject and edits made by each volunteer will be saved and passed to the next.

---


# Step 1 — Configure the Reducer for Collaborative Classifications

The first step is to configure the reducer, which must include the following key-value pairs:

```yaml
average_type: last
collab: True
```

- **`average_type: last`**
  Ensures that the current classification uses only the **most recent (last)** classification made by a previous volunteer, rather than computing an average across all classifications.
  This is critical for a collaborative workflow where each participant builds upon or edits the last version.

- **`collab: True`**
  Enables collaborative mode.
  When `collab` is set to `True`, the system will:
  - Display the previous volunteer’s freehand classification to the current volunteer.
  - Allow the current volunteer to **edit** that freehand drawing directly.

The following keyword arguments (`kwargs`) are **required by the front end**, but will **default automatically** if not specified:

| Parameter   | Default Value   | Description |
|--------------|----------------|--------------|
| `stepKey`    | `'S0'`          | The classification step key. |
| `taskIndex`  | `0`             | Index of the task in the workflow. |
| `toolType`   | `'freehandLine'` | The drawing tool type used for freehand classifications. |

These parameters can be overridden if your workflow uses different step keys, task indices, or drawing tools.

The following parameters are also part of the reducer configuration, but are not specific to the collaborative workflow.
They can be tuned as needed for your project:

| Parameter     | Description |
|----------------|-------------|
| `eps`          | Distance threshold used for clustering (DBSCAN parameter). |
| `min_samples`  | Minimum number of samples required to form a cluster (DBSCAN parameter). |

The other parameters are handled internally by the aggregation software. These are:


| Parameter   | Description |
|-------------|-------------|
| `taskKey`   |             |
| `taskType`  |             |
| `toolIndex` |             |
| `frame`     |             |
| `markID`    |             |
| `pathX`     |             |
| `pathY`     |             |

An example of the reducer configuration file can be seen below:

![Caesar settings screenshot](./_static/Reducer_config_collab_workflow.png)

With this configuration:

- Only the last classification is used as the base for the next volunteer.
- The collaborative workflow is active.
- `min_samples` is set to 1 and `eps` to 0.5
- The rest of the parameters will be their default values as they are not provided.

---

# Step 2 — Database Setup on Caesar

Once the reducer has been configured for collaborative classifications, the next step is to set up the extractor and reducer on [Caesar](https://caesar.zooniverse.org/).

## Extractor Setup

The extractor defines how classification data is pulled from the Zooniverse workflow. On Caesar, navigate to Workflows, then your project. Under the Extractors tab, select Create Extractor -> External:

![extractor_setup](./_static/extractor_setup.png)

Next, we need to fill out the fields. The minimum workflow version number can be found from the Zooniverse front end. On the project builder, navigate to the Workflows tab and select the workflow you're setting up the collaborative workflow for. The version is shown in the second section:

![workflow_version](./_static/workflow_version.png)

In this example, the workflow version is 4.5.

Finally, we need to construct the URL. You can build the extractor URL using Python’s `urllib.parse` library. For our example above, we use:

```python
import urllib.parse

# Insert any other parameters that aren't default values
config = {'task': 'T0', 'tool': [0]}
print(urllib.parse.urlencode(config))
```

This will output the encoded configuration string:

```
task=T0&tool=%5B0%5D
```

Use this to construct the full extractor URL:

```
https://aggregation-caesar.zooniverse.org/extractors/polygon_extractor?task=T0&tool=%5B0%5D
```

This URL tells Caesar to use the polygon extractor, applying it to task T0 and tool 0.

![Extractor](./_static/ExternalExtractor.png)

## Reducer Setup

Next, we need to set up the reducer configuration that processes the extracted data. Navigate to the Reducers tab, then select Create Reducer -> External:

![reducer_setup](./_static/reducer_setup.png)

We then need to fill out the fields as follows:

- **Key:** `machineLearnt`
- **Version:**
  Enter the same version number used for the **minimum workflow version** in the extractor setup. For this example it is 4.5.

Again, to construct the URL, we use Python’s `urllib.parse` to encode the configuration parameters:

```python
import urllib.parse

# Insert any other parameters that aren't default values
config = {'average_type': 'last', 'min_samples': [1], 'collab': True}
print(urllib.parse.urlencode(config))
```

This will output the encoded configuration string:

```
average_type=last&min_samples=%5B1%5D&collab=True
```

Use this to construct the full reducer URL:

```
https://aggregation-caesar.zooniverse.org/reducers/polygon_reducer?average_type=last&min_samples=%5B1%5D&collab=True
```

This URL defines a polygon reducer that:
- Uses the last classification (`average_type=last`)
- Applies clustering with `min_samples = 1`
- Enables collaborative editing (`collab=True`)

![Reducer](./_static/ExternalReducer.png)

Once both the ExternalExtractor and ExternalReducer are configured, verify that the URLs resolve correctly when entered in a browser. You should see a string confirming the extractor and reducer endpoints.

![url_extractor](./_static/URL_extractor.png)

![url_reducer](./_static/URL_reducer.png)

---


# Step 3 — Front-End Configuration

The final step is to configure the **Zooniverse front end** so that it can communicate with Caesar and display collaborative data correctly.

In the project builder on Zooniverse, go to the Workflow tab and scroll to the bottom.
You will see a checkbox labelled Enable Caesar Data Fetching, which needs to be checked for the collaborative workflow to work.

![Caesar_Data_Fetching](./_static/CaesarDataFetching.png)

Enabling Caesar Data Fetching allows the Zooniverse front end to retrieve extractor and reducer outputs directly from Caesar.
This is required for the collaborative workflow to display the previous volunteer’s classification to the next participant.

Once Caesar data fetching is enabled and configured:

- Volunteers should now see the **previous volunteer’s freehand drawing** when classifying.
- Any edits made will be saved and passed forward to the next volunteer.
- The reducer will continue to use only the **latest classification** (`average_type: last`) to maintain a continuous chain of collaboration.

---
